In [ ]:
import h5py, os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm

In [ ]:
import sys; sys.path.append("../../offline/")
import sparse, geom, utils

In [ ]:
path_to_data = "../../data/sparse/" # scratch/sparse on Maxwell
path_to_aux = "../../data/aux/" # scratch/benedikt/aux on Maxwell
path_to_geometry = "../../geometry/"
path_to_recons = "../../data/recons/"

## Load detector file from 2160

In [ ]:
emc_detector = path_to_recons + "det/det_2160_lowq7.h5"
with h5py.File(emc_detector, "r") as f:
    mask = f['mask'][:].reshape((4,128,128))
new_mask = mask.copy()

## Include bad-pixels (identified during 2DEMC classification)

In [ ]:
# from cell 1
new_mask[0,107:108+1,16:107+1] = 2
new_mask[0,109,106] = 2
new_mask[0,109,108] = 2
new_mask[0,114,109] = 2
new_mask[0,116,108] = 2
new_mask[0,120,107] = 2
new_mask[0,123,104] = 2
new_mask[0,125,103] = 2
new_mask[0,125,106] = 2

In [ ]:
# from cell 2
new_mask[3,125,57:58+1] = 2
new_mask[3,126,58:59+1] = 2
new_mask[3,124,42] = 2
new_mask[3,122,41:42+1] = 2
new_mask[3,114:118+1,37] = 2
new_mask[3,114:117+1,36] = 2
new_mask[3,115,35] = 2
new_mask[3,111,35] = 2
new_mask[3,109,34:36+1] = 2
new_mask[3,106:108+1,33:36+1] = 2
new_mask[3,105,34] = 2

In [ ]:
# from cell 12
new_mask[3,110,35] = 2
new_mask[3,90,27] = 2
new_mask[3,68,40] = 2
new_mask[3,20,67] = 2
new_mask[3,34,116] = 2
new_mask[3,105,103] = 2

In [ ]:
# bright outliers
new_mask[1,27,34] = 2
new_mask[1,45,47] = 2
new_mask[0,112,105]  = 2

In [ ]:
# more bad-pixels
new_mask[1,122,20:23] = 2
new_mask[1,124,20:23] = 2

In [ ]:
# Halo from direct beam
new_mask[1,124,1:10] = 2
new_mask[1,125,1:12] = 2
new_mask[1,126,1:14] = 2

## Plot new mask

In [ ]:
plt.figure(figsize=(5,5), dpi=100)
plt.axis('off')
plt.imshow(utils.assemble(np.transpose(new_mask, axes=(0,2,1)), path_to_geometry + "b2_lowq.geom"), cmap='gray')
plt.savefig("../../plots/lowq_mask1.png", bbox_inches='tight')
plt.show()

## Save mask in new detector file (version 1)

In [ ]:
new_detector = path_to_recons + "det/det_2145_lowq1.h5"
os.system("cp %s %s" %(emc_detector, new_detector))
with h5py.File(new_detector, "a") as f:
    f['mask'][:] = new_mask.ravel()

## Include statistical outliers from long dark (run 597)

In [ ]:
new_mask2 = new_mask.copy()
masks = []
for m in range(16):
    with h5py.File(path_to_recons + 'det/r0597/r0597_AGIPD%02d_v01_mask.h5' %m, 'r') as f:
        masks.append(np.bool8(f['mask'][:]))
masks = np.array(masks)
combined_mask = np.transpose(masks.all(axis=1),axes=(0,2,1))

In [ ]:
new_mask2[0][combined_mask[3, :,-128:]==0] = 2
new_mask2[1][combined_mask[4, :,-128:]==0] = 2
new_mask2[2][combined_mask[8, :,-128:]==0] = 2
new_mask2[3][combined_mask[15,:,-128:]==0] = 2

## Plot new mask

In [ ]:
plt.figure(figsize=(5,5), dpi=100)
plt.axis('off')
plt.imshow(utils.assemble(np.transpose(new_mask2, axes=(0,2,1)), path_to_geometry + "b2_lowq.geom"), cmap='gray')
plt.savefig("../../plots/lowq_mask2.png", bbox_inches='tight')
plt.show()

## Save mask in new detector file (version 2)

In [ ]:
new_detector = path_to_recons + "det/det_2145_lowq2.h5"
os.system("cp %s %s" %(emc_detector, new_detector))
with h5py.File(new_detector, "a") as f:
    f['mask'][:] = new_mask2.ravel()

## Include circular mask to block high-q

In [ ]:
new_mask3 = new_mask.copy()
with h5py.File(emc_detector, "r") as f:
    qx = f['qx'][:].reshape((4,128,128))
    qy = f['qy'][:].reshape((4,128,128))

In [ ]:
qr = np.sqrt(qx**2 + qy**2)
new_mask3[qr>70] = 2

In [ ]:
plt.imshow(utils.assemble(np.transpose(qr<70,axes=(0,2,1)), geomfile='../../geometry/b2_lowq.geom'))
plt.show()

In [ ]:
new_detector = path_to_recons + "det/det_2145_lowq3.h5"
os.system("cp %s %s" %(emc_detector, new_detector))
with h5py.File(new_detector, "a") as f:
    f['mask'][:] = new_mask3.ravel()

## Prepare mask for merging with LCLS data

In [ ]:
new_mask4 = new_mask.copy()
with h5py.File(emc_detector, "r") as f:
    qx = f['qx'][:].reshape((4,128,128))
    qy = f['qy'][:].reshape((4,128,128))
    qz = f['qz'][:].reshape((4,128,128))
    print(list(f))
    print(f['detd'][...])
    print(f['ewald_rad'][...])
    print(f['corr'][...])

In [ ]:
qr = np.sqrt(qx**2 + qy**2)
new_mask4[qr>50] = 2
new_mask4[(qr>25) & (qr<=50) & (new_mask4 == 0)] = 1

In [ ]:
active = utils.assemble(np.transpose(np.ones((4,128,128)),axes=(0,2,1)),geomfile='../../geometry/b2_lowq.geom')
masked = utils.assemble(np.transpose(new_mask4,axes=(0,2,1)),geomfile='../../geometry/b2_lowq.geom')

In [ ]:
cmap = cm.get_cmap('hot')
cmap.set_bad('0.90')
cmap.set_under('1.0')
mimg = np.ma.masked_array(masked, mask=1-active)
mimg.data[mimg.data==0] = 1e-10
plt.figure(figsize=(15,10), dpi=200)
plt.imshow(mimg, cmap=cmap)
plt.gca().add_patch(plt.Circle((171,177),88, lw=1, fill=0, color='b'))
plt.gca().add_patch(plt.Circle((171,177),44, lw=1, fill=0, color='g'))
#plt.axis('off')
plt.grid(color='y', ls=':')
#plt.colorbar()
plt.show()

In [ ]:
new_detector = path_to_recons + "det/det_2145_lowq4.h5"
os.system("cp %s %s" %(emc_detector, new_detector))
with h5py.File(new_detector, "a") as f:
    f['mask'][:] = new_mask4.ravel()

## Same as lowq4 but with all q's inside 50

In [ ]:
new_mask5 = new_mask.copy()
with h5py.File(emc_detector, "r") as f:
    qx = f['qx'][:].reshape((4,128,128))
    qy = f['qy'][:].reshape((4,128,128))

In [ ]:
qr = np.sqrt(qx**2 + qy**2)
new_mask5[qr>50] = 2

In [ ]:
new_detector = path_to_recons + "det/det_2145_lowq5.h5"
os.system("cp %s %s" %(emc_detector, new_detector))
with h5py.File(new_detector, "a") as f:
    f['mask'][:] = new_mask5.ravel()